El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

In [119]:


# Manipulación de datos
import pandas as pd
import numpy as np

# Visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesamiento de datos
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Modelos
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# Evaluación del modelo
from sklearn.metrics import mean_squared_error

# Medición del tiempo de ejecución
import time

# Gestión de advertencias
import warnings
warnings.filterwarnings('ignore')

## Preparación de datos

In [120]:
# Carga de dataset

df = pd.read_csv('car_data.csv')

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [122]:
df.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [123]:
print(df.head())

        DateCrawled  Price VehicleType  RegistrationYear Gearbox  Power  \
0  24/03/2016 11:52    480         NaN              1993  manual      0   
1  24/03/2016 10:58  18300       coupe              2011  manual    190   
2  14/03/2016 12:52   9800         suv              2004    auto    163   
3  17/03/2016 16:54   1500       small              2001  manual     75   
4  31/03/2016 17:25   3600       small              2008  manual     69   

   Model  Mileage  RegistrationMonth  FuelType       Brand NotRepaired  \
0   golf   150000                  0    petrol  volkswagen         NaN   
1    NaN   125000                  5  gasoline        audi         yes   
2  grand   125000                  8  gasoline        jeep         NaN   
3   golf   150000                  6    petrol  volkswagen          no   
4  fabia    90000                  7  gasoline       skoda          no   

        DateCreated  NumberOfPictures  PostalCode          LastSeen  
0  24/03/2016 00:00               

In [124]:

df_names = ['date_crawled', 'price', 'vehicle_type', 'registration_year', 'gear_box', 'power', 
 'model', 'mileage', 'registration_month', 'fuel_type', 'brand', 'not_repaired', 
 'date_created', 'number_of_pictures', 'postal_code', 'last_seen']
df.columns = df_names

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        354369 non-null  object
 1   price               354369 non-null  int64 
 2   vehicle_type        316879 non-null  object
 3   registration_year   354369 non-null  int64 
 4   gear_box            334536 non-null  object
 5   power               354369 non-null  int64 
 6   model               334664 non-null  object
 7   mileage             354369 non-null  int64 
 8   registration_month  354369 non-null  int64 
 9   fuel_type           321474 non-null  object
 10  brand               354369 non-null  object
 11  not_repaired        283215 non-null  object
 12  date_created        354369 non-null  object
 13  number_of_pictures  354369 non-null  int64 
 14  postal_code         354369 non-null  int64 
 15  last_seen           354369 non-null  object
dtypes:

In [125]:
# Chqueo de duplicados

print(df.duplicated().sum())

df = df.drop_duplicates().reset_index(drop=True)

print(df.duplicated().sum())

262
0


In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354107 entries, 0 to 354106
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        354107 non-null  object
 1   price               354107 non-null  int64 
 2   vehicle_type        316623 non-null  object
 3   registration_year   354107 non-null  int64 
 4   gear_box            334277 non-null  object
 5   power               354107 non-null  int64 
 6   model               334406 non-null  object
 7   mileage             354107 non-null  int64 
 8   registration_month  354107 non-null  int64 
 9   fuel_type           321218 non-null  object
 10  brand               354107 non-null  object
 11  not_repaired        282962 non-null  object
 12  date_created        354107 non-null  object
 13  number_of_pictures  354107 non-null  int64 
 14  postal_code         354107 non-null  int64 
 15  last_seen           354107 non-null  object
dtypes:

Después de eliminar los duplicados y cambiar los nombres de las columnas para una mejor visualización de las mimas, observo que hay una buena cantidad de entradas nulas en algunas columnas, por lo que a continuación trabajare en esto.

In [127]:
# Función de rellenado de datos nulos

'''
Toma los nombres de las columnas de un dataset dado y itera sobre cada columna 
chequeando que no haya entradas nulas y que esa columna sea tipo object,
si se cumplen las condiciones rellena esa columna con el string 'Unknown'

    Args:
        dataset : Simple dataFrame de pandas
    Returns:
        datase : Retorna el dataset con las columnas string rellenas
    """
'''

def null_full(dataset):
    for i in dataset.columns:
        if dataset[i].isnull().count() > 0 and dataset[i].dtype == object:
            dataset[i] = dataset[i].fillna('Unknown')
        
    return dataset            

In [128]:
# Rellenado de entradas nulas

df = null_full(df)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354107 entries, 0 to 354106
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        354107 non-null  object
 1   price               354107 non-null  int64 
 2   vehicle_type        354107 non-null  object
 3   registration_year   354107 non-null  int64 
 4   gear_box            354107 non-null  object
 5   power               354107 non-null  int64 
 6   model               354107 non-null  object
 7   mileage             354107 non-null  int64 
 8   registration_month  354107 non-null  int64 
 9   fuel_type           354107 non-null  object
 10  brand               354107 non-null  object
 11  not_repaired        354107 non-null  object
 12  date_created        354107 non-null  object
 13  number_of_pictures  354107 non-null  int64 
 14  postal_code         354107 non-null  int64 
 15  last_seen           354107 non-null  object
dtypes:

Ahora observo en los datos que algunas son fechas con horas, estas las trabajare con un función de la misma forma para convertirlos a datetime

In [129]:
# Convertir las columnas a tipo datetime

df['date_crawled'] = pd.to_datetime(df['date_crawled'], format='%d/%m/%Y %H:%M')

df['date_created'] = pd.to_datetime(df['date_created'], format='%d/%m/%Y %H:%M')

df['last_seen'] = pd.to_datetime(df['last_seen'], format='%d/%m/%Y %H:%M')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354107 entries, 0 to 354106
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   date_crawled        354107 non-null  datetime64[ns]
 1   price               354107 non-null  int64         
 2   vehicle_type        354107 non-null  object        
 3   registration_year   354107 non-null  int64         
 4   gear_box            354107 non-null  object        
 5   power               354107 non-null  int64         
 6   model               354107 non-null  object        
 7   mileage             354107 non-null  int64         
 8   registration_month  354107 non-null  int64         
 9   fuel_type           354107 non-null  object        
 10  brand               354107 non-null  object        
 11  not_repaired        354107 non-null  object        
 12  date_created        354107 non-null  datetime64[ns]
 13  number_of_pictures  354107 no

In [130]:
# Crear una instancia de StandardScaler
scaler = StandardScaler()

# Identificar columnas numéricas
numeric_columns = ['registration_year', 'power', 'mileage', 'registration_month', 'number_of_pictures']

# Ajustar el escalador y transformar las columnas numéricas
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# Verificar los cambios
print(df.head())



         date_crawled  price vehicle_type  registration_year gear_box  \
0 2016-03-24 11:52:00    480      Unknown          -0.124476   manual   
1 2016-03-24 10:58:00  18300        coupe           0.074945   manual   
2 2016-03-14 12:52:00   9800          suv          -0.002607     auto   
3 2016-03-17 16:54:00   1500        small          -0.035844   manual   
4 2016-03-31 17:25:00   3600        small           0.041708   manual   

      power    model   mileage  registration_month fuel_type       brand  \
0 -0.579679     golf  0.574787           -1.533319    petrol  volkswagen   
1  0.420770  Unknown -0.084730           -0.191641  gasoline        audi   
2  0.278601    grand -0.084730            0.613366  gasoline        jeep   
3 -0.184765     golf  0.574787            0.076695    petrol  volkswagen   
4 -0.216358    fabia -1.008053            0.345031  gasoline       skoda   

  not_repaired date_created  number_of_pictures  postal_code  \
0      Unknown   2016-03-24             

In [131]:
print(df.head(10))

         date_crawled  price vehicle_type  registration_year gear_box  \
0 2016-03-24 11:52:00    480      Unknown          -0.124476   manual   
1 2016-03-24 10:58:00  18300        coupe           0.074945   manual   
2 2016-03-14 12:52:00   9800          suv          -0.002607     auto   
3 2016-03-17 16:54:00   1500        small          -0.035844   manual   
4 2016-03-31 17:25:00   3600        small           0.041708   manual   
5 2016-04-04 17:36:00    650        sedan          -0.102318   manual   
6 2016-04-01 20:48:00   2200  convertible          -0.002607   manual   
7 2016-03-21 18:54:00      0        sedan          -0.268503   manual   
8 2016-04-04 23:42:00  14500          bus           0.108182   manual   
9 2016-03-17 10:53:00    999        small          -0.069081   manual   

      power    model   mileage  registration_month fuel_type       brand  \
0 -0.579679     golf  0.574787           -1.533319    petrol  volkswagen   
1  0.420770  Unknown -0.084730           -0.

Hay algunas columnas que no son necesarias a la hora de predecir el coste de un coche como por ejemplo DateCrawled, NumberOfPictures, PostalCode o LastSeen por que a continuación las eliminare

In [132]:
df_prepared = df.drop(['date_crawled', 'number_of_pictures', 'last_seen', 'postal_code'], axis=1)
df

,date_crawled,price,vehicle_type,registration_year,gear_box,power,model,mileage,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen
0,2016-03-24 11:52:00,480,Unknown,-0.124476,manual,-0.579679,golf,0.574787,-1.533319,petrol,volkswagen,Unknown,2016-03-24,0.0,70435,2016-04-07 03:16:00
1,2016-03-24 10:58:00,18300,coupe,0.074945,manual,0.420770,Unknown,-0.084730,-0.191641,gasoline,audi,yes,2016-03-24,0.0,66954,2016-04-07 01:46:00
2,2016-03-14 12:52:00,9800,suv,-0.002607,auto,0.278601,grand,-0.084730,0.613366,gasoline,jeep,Unknown,2016-03-14,0.0,90480,2016-04-05 12:47:00
3,2016-03-17 16:54:00,1500,small,-0.035844,manual,-0.184765,golf,0.574787,0.076695,petrol,volkswagen,no,2016-03-17,0.0,91074,2016-03-17 17:40:00
4,2016-03-31 17:25:00,3600,small,0.041708,manual,-0.216358,fabia,-1.008053,0.345031,gasoline,skoda,no,2016-03-31,0.0,60437,2016-04-06 10:17:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354102,2016-03-21 09:50:00,0,Unknown,0.008471,manual,-0.579679,colt,0.574787,0.345031,petrol,mitsubishi,yes,2016-03-21,0.0,2694,2016-03-21 10:42:00
354103,2016-03-14 17:48:00,2200,Unknown,0.008471,Unknown,-0.579679,Unknown,-2.854701,-1.264983,Unknown,sonstige_autos,Unknown,2016-03-14,0.0,39576,2016-04-06 00:46:00
354104,2016-03-05 19:56:00,1199,convertible,-0.046923,auto,-0.047862,fortwo,-0.084730,-0.728312,petrol,smart,no,2016-03-05,0.0,26135,2016-03-11 18:17:00
354105,2016-03-19 18:57:00,9200,bus,-0.091239,manual,-0.042596,transporter,0.574787,-0.728312,gasoline,volkswagen,no,2016-03-19,0.0,87439,2016-04-07 07:15:00


In [133]:
# Dvidir el dataset

df_train_valid, df_test = train_test_split(df_prepared, test_size=0.2, random_state=12345)

df_train, df_valid = train_test_split(df_train_valid, test_size=0.25)


# Verificamos los tamaños de cada conjunto
print("Tamaño del conjunto de entrenamiento:", df_train.shape)
print("Tamaño del conjunto de validación:", df_valid.shape)
print("Tamaño del conjunto de prueba:", df_test.shape)

Tamaño del conjunto de entrenamiento: (212463, 12)
Tamaño del conjunto de validación: (70822, 12)
Tamaño del conjunto de prueba: (70822, 12)


In [134]:
# features y targets

# Entrenamiento
features_train = df_train.drop('price', axis=1)
targets_train = df_train['price']

# Validación
features_valid = df_valid.drop('price', axis=1)
targets_valid = df_valid['price']

Estos dato estan preparados para poder ser usados en el entrenamiento de la mayoria de modelos, sin embargo hay modelos que no aceptan datos categoricos, sino que necesitan OHE, además librerias como XGBoost no admite tampoco columnas tipo datetime y es lo que relizare a continuación 

### Dataset especial para XGBoost y modelos que requieren OHE

In [135]:
# Copia del dataset ya procesado
data_ohe = df_prepared.copy()

data_ohe.head()

,price,vehicle_type,registration_year,gear_box,power,model,mileage,registration_month,fuel_type,brand,not_repaired,date_created
0,480,Unknown,-0.124476,manual,-0.579679,golf,0.574787,-1.533319,petrol,volkswagen,Unknown,2016-03-24
1,18300,coupe,0.074945,manual,0.420770,Unknown,-0.084730,-0.191641,gasoline,audi,yes,2016-03-24
2,9800,suv,-0.002607,auto,0.278601,grand,-0.084730,0.613366,gasoline,jeep,Unknown,2016-03-14
3,1500,small,-0.035844,manual,-0.184765,golf,0.574787,0.076695,petrol,volkswagen,no,2016-03-17
4,3600,small,0.041708,manual,-0.216358,fabia,-1.008053,0.345031,gasoline,skoda,no,2016-03-31


In [136]:
categorical_columns = data_ohe[['vehicle_type', 'gear_box', 'model', 'fuel_type', 'brand', 'not_repaired']]

# Separar por cantidad de tiempo la columna date_column
data_ohe['year'] = data_ohe['date_created'].dt.year
data_ohe['month'] = data_ohe['date_created'].dt.month
data_ohe['day'] = data_ohe['date_created'].dt.day
data_ohe = data_ohe.drop('date_created', axis=1)


data_ohe = pd.get_dummies(data_ohe,drop_first=True).astype(int)

# Crear una instancia de StandardScaler
scaler = StandardScaler()
data_ohe[['day', 'month', 'year']] = scaler.fit_transform(data_ohe[['day', 'month', 'year']])

data_ohe

,price,registration_year,power,mileage,registration_month,year,month,day,vehicle_type_bus,vehicle_type_convertible,...,brand_smart,brand_sonstige_autos,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,not_repaired_no,not_repaired_yes
0,480,0,0,0,-1,0.008426,-0.425641,0.892231,0,0,...,0,0,0,0,0,0,1,0,0,0
1,18300,0,0,0,0,0.008426,-0.425641,0.892231,0,0,...,0,0,0,0,0,0,0,0,0,1
2,9800,0,0,0,0,0.008426,-0.425641,-0.203918,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1500,0,0,0,0,0.008426,-0.425641,0.124927,0,0,...,0,0,0,0,0,0,1,0,1,0
4,3600,0,0,-1,0,0.008426,-0.425641,1.659536,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354102,0,0,0,0,0,0.008426,-0.425641,0.563387,0,0,...,0,0,0,0,0,0,0,0,0,1
354103,2200,0,0,-2,-1,0.008426,-0.425641,-0.203918,0,0,...,0,1,0,0,0,0,0,0,0,0
354104,1199,0,0,0,0,0.008426,-0.425641,-1.190452,0,1,...,1,0,0,0,0,0,0,0,1,0
354105,9200,0,0,0,0,0.008426,-0.425641,0.344157,1,0,...,0,0,0,0,0,0,1,0,1,0


In [137]:
# Dvidir el dataset

df_train_valid_ohe, df_test_ohe = train_test_split(data_ohe, test_size=0.2, random_state=12345)

df_train_ohe, df_valid_ohe = train_test_split(df_train_valid_ohe, test_size=0.25, random_state=12345)


# Verificamos los tamaños de cada conjunto
print("Tamaño del conjunto de entrenamiento:", df_train_ohe.shape)
print("Tamaño del conjunto de validación:", df_valid_ohe.shape)
print("Tamaño del conjunto de prueba:", df_test_ohe.shape)

Tamaño del conjunto de entrenamiento: (212463, 316)
Tamaño del conjunto de validación: (70822, 316)
Tamaño del conjunto de prueba: (70822, 316)


In [142]:
# features y targets

# Entrenamiento
features_train_ohe = df_train_ohe.drop('price', axis=1)
targets_train_ohe = df_train_ohe['price']

# Validación
features_valid_ohe = df_valid_ohe.drop('price', axis=1)
targets_valid_ohe = df_valid_ohe['price']

## Entrenamiento del modelo 

#### Modelo de regresión lineal

In [144]:
model_reg = LinearRegression()

model_reg.fit(features_train_ohe, targets_train_ohe)




In [145]:
# Predicciónes
predictions_valid_ohe = model_reg.predict(features_valid_ohe)

# Calcular RMSE
rmse = np.sqrt(mean_squared_error(targets_valid_ohe, predictions_valid_ohe))

print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 3290.328231676506


## Análisis del modelo

# Lista de control

- [x]  Jupyter Notebook está abierto
- [ ]  El código no tiene errores- [ ]  Las celdas con el código han sido colocadas en orden de ejecución- [ ]  Los datos han sido descargados y preparados- [ ]  Los modelos han sido entrenados
- [ ]  Se realizó el análisis de velocidad y calidad de los modelos